In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
qsos = pd.read_csv('qso_full_sample.csv', header=1)
qsos

,specobjid,subclass,z,oiii_5007_flux,h_alpha_flux,h_beta_flux,nii_6584_flux
0,409913086674233344,AGN,0.027287,1623.99100,777.0163,184.89060,859.6522
1,412190450418477056,STARFORMING,0.029046,100.78620,1753.8930,371.54320,818.9180
2,413269895623501824,STARFORMING,0.061882,78.43416,612.8790,115.02330,497.0024
3,417743534917969920,STARFORMING BROADLINE,0.027417,68.33032,575.3583,145.75290,388.0303
4,661020102417737728,NaN,0.026416,78.46451,108.7614,39.84615,104.7426
...,...,...,...,...,...,...,...
980,2816010150828075008,STARBURST,0.031757,1169.68800,3050.7800,903.23750,949.1982
981,2817155293799016448,AGN BROADLINE,0.022155,107.54270,542.0471,121.83740,430.3494
982,3343996467592325120,BROADLINE,0.034936,80.73515,144.3445,39.16608,141.3919
983,3344069859993479168,STARFORMING BROADLINE,0.034969,61.79113,530.8751,85.17157,311.0701


In [8]:
qsos.shape

(985, 7)

In [9]:
qsos.columns

Index(['specobjid', 'subclass', 'z', 'oiii_5007_flux', 'h_alpha_flux',
       'h_beta_flux', 'nii_6584_flux'],
      dtype='object')